<a href="https://colab.research.google.com/github/SeongwonTak/TIL_swtak/blob/master/DLScratch1Study/DLScratch_ch5_prepare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 밑바닥부터 시작하는 딥러닝_5장 스터디 준비
어느 덧, 스터디 4주차.  5장 스터디를 준비한 내용을 정리한다.
(내용이 상당히 어렵더라..)

## 5.1 계산 그래프
### 계산그래프란?
- 계산과정을 그래프로 표현
- 여기서 말하는 그래프는 node와 edge로 표현하는 그림.
- 계산과정을 왼쪽에서 오른쪽으로 진행하면 (forward propagation)
- 계산과정을 오른쪽에서 왼쪽으로 역방향 진행하면 (backword propagation)
  - 역전파는 미분을 계산할때 중요한 역할을 한다.(역전파가 미분을 전달하게된다)

**질문사항, 대체 왜 역함수가 아닌 미분을 역전파로 전달하는거를 생각하는걸까?**

### 계산그래프의 예시



## 5.2 chain rule
- 계산 그래프의 역전파
순전파때의 계산의 미분을 구한다.


### chain rule(미분 표현)
$z=f(t)$
$t=f(x)$
일 경우,
$$\frac{\partial z}{\partial x}=\frac{\partial z}{\partial t}\frac{\partial t}{\partial x}$$

## 5.3~5.4 덧셈노드와 곱셈노드의 역전파 

### 덧셈노드
만일 $z = x+y$라면,
$$ \frac{\partial z}{\partial x}=1$$
$$ \frac{\partial z}{\partial y}=1$$
이므로, 덧셈 노드에 대해서 역전파를 보낼 경우에는 같은 값을 그대로 다음 노드에 전달하게 된다.


In [ ]:
class AddLayer:
  def __init__(self):
    pass
  
  def forward(self, x, y):
    out = x + y
    return out

  def backward(self, dout):
    dx = dout * 1
    dy = dout * 1
    return dx, dy

## 곱셈노드
만일 $z = xy$라면,
$$ \frac{\partial z}{\partial x}=y$$
$$ \frac{\partial z}{\partial y}=x$$
이다.즉, 곱셈 노드 역전파는 순전파 때의 입력 신호들을 **서로 바꾼 값을 곱해서** 하류로 보낸다.

In [ ]:
class MulLayer:
  def __init__(self):
    self.x = None
    self.y = None
  
  def forward(self, x, y):
    self.x = x
    self.y = y
    out = x * y
  
  def backward(self, dout):
    dx = dout * self.y
    dy = dout * self.x

    return dx, dy

### 예시

## 5.5 활성화 함수 계층 구현




### ReLU
ReLU 함수를 미분하면, 0보다 클때는 1, 아닐때는 0으로 나올것이다.

In [ ]:
class Relu:
  def __init__(self):
    self.mask = None
  
  def forward(self, x):
    self.mask = (x <= 0)
    out = x.copy()
    out[self.mask] = 0

    return out

  def backward(self, dout):
    dout[self.mask] = 0
    dx = dout
  
    return dx


### sigmoid
sigmoid의 경우는 4단계로 나누어서 함수 계산을 한다.

x -> -x -> exp(-x) -> 1+exp(-x) -> sigmoid

 즉 * -1,  exp,  +1,  역수의 과정으로 연산을 한 것이며 각 함수를 미분하여 역으로 돌려주는 과정을 역전파로 구한다.

 - 역전파 1단계 : $y=\frac{1}{x}$의 역전파 구하기
 - 역전파 2단계 : 덧셈의 역전파
 - 역전파 3단계 : $y = exp(x)$의 역전파 구하기
 - 역전파 4단계 : 곱셈의 역전파

 위와 동일한 식으로, 미분을 구하여 곱하는 형태로 역전파를 구한다.

In [ ]:
class sigmoid:
  def __init__(self):
    self.out = None
  
  def forward(self, x):
    out = 1 / (1 + np.exp(-x))
    self.out = out

    return out
  
  def backward(self, dout):
    dx = dout* (1.0 - self.out) * self.out
    
    return dx

## 5.6 Affine 계층과 Softmax 계층



### Affine 계층
앞에서 배운 신경망의 순전파는 행렬의 곱이다.
이에 대한 역전파를 고려하자.

$X \cdot W$의 미분을 고려하는 것으로 충분하다.
(상수항의 덧셈부분, 덧셈의 역전파는 그대로 전달한다는 사실을 잊지 말자)

$$\frac{\partial L}{\partial X}=\frac{\partial L}{\partial Y} \cdot W^{T} $$

$$\frac{\partial L}{\partial W}=X^{T}\cdot \frac{\partial L}{\partial Y}  $$

역전파 또한 배열로 주어진다.

Batch Model을 고려해도 결과에는 변화가 없다.

**Remark. 행렬의 미분에 대한 내용 확인 필요**

In [ ]:
class Affine:
  def __init__(self, W, b):
    self.W = W
    self.b = b
    self.x = None
    self.dW = None
    self.db = None

  def forward(self, x):
    self.x = x
    out = np.dot(x, self.W) + self.b

    return uot

  def backward(self, dout):
    dx = np.dot(dout, self.W.T)
    self.dw = np.dot(self.x.T, dout)
    self.db = np.sum(dout, axis=0)

    return dx

### Softmax 역전파
(증명은 추후 기술)
출력이 $(y_{1},y_{2},y_{3})$이고 정답이 $(t_{1},t_{2},t_{3})$일때, 역전파의 결과는 두 벡터의 차이이다.



In [ ]:
class SoftmaxWithLoss:
  def __init__(self):
    self.loss = None
    self.y = None
    self.t = None
  
  def forward(self, x, t):
    self.t = t
    self.y = softmax(x)
    self.loss = cross_entropy_error(self.y, self.t)
    return self.loss
  
  def backward(self, dout = 1):
    batch_size = self.t.shape[0]
    dx = (self.y - self.t)/batch_size

    return dx

## 5.7 오차 역전파법의 구현

- 신경망 학습의 전체 그림은 동일함
미니배치 -> 기울기산출 -> 매개변수의 갱신을 반복.

In [ ]:
# 오차역전파법이 적용된 신경망
import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

class TwoLayerNet:
  def __init__(self, input_size, hidden_size, output_size,
               weight_init_std = 0.01):
    self.params = {}
    self.params['W1'] = weigit_init_std * np.random.randn(input_size, hidden_size)
    self.params['b1'] = np.zeros(hidden_size)
    self.params['W2'] = weigit_init_std * np.random.randn(input_size, hidden_size)
    self.params['b2'] = np.zeros(output_size)

    # 계층의 생성
    self.layers = OrderedDict()
    self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
    self.layers['Relu1'] = Relu()
    self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

    self.lastLayer = SoftmaxWithLoss()

  def predict(self, x):
    for layer in self.layers.values():
      x = layer.forward(x)

    return x

  def loss(self, x, t):
    y = self.predict(x)
    return self.lastLayer.forward(y, t)

  def accuracy(self, x, t):
    y = self.predict(x)
    y = np.argmax(y, axis = 1)
    if t.ndim != 1 : t = np.argmax(t, axis = 1)

    accuracy = np.sum(y == t) / float(x.shape[0])
    return accuracy

  def numerical_gradient(self, x, t):
    loss_W = lambda W : self.loss(x, t)

    grads = {}
    grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
    grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
    grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
    grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
    return grads

  def gradient(self, x, t):
    self.loss(x, t)  # 순전파

    dout = 1
    dout = self.lastLayer.backward(dout)

    layers = list(self.layers.values())
    layers.reverse()
    for layer in layers:
      dout = layer.backward(dout)

    grads = {}
    grads['W1'] = self.layers['Affine1'].dW
    grads['b1'] = self.layers['Affine1'].db
    grads['W2'] = self.layers['Affine2'].dW
    grads['b2'] = self.layers['Affine2'].db

    return grads